# LAB 5: Image search using CLIP

## Overview
1. The code will run encode all pictures in `datasetFolder_path` into embeddings
2. The user interface will receive an input image, encode it, and find similarity between an input image embeddings and dataset embeddings
3. The dataset image that have the highest similarity score will be shown on the output box.
## Before running the code
1. set the `datasetFolder_path` into the folder you want dataset image to be.
2. find the images you want to be in your dataset.
3. find the image that is not include in the dataset to test the algorithm.

In [1]:
# install library

! pip install torch ftfy regex tqdm
! pip install openai-clip
! pip install gradio

In [24]:
# import essential library

import clip
from PIL import Image
import torch
import os
from tqdm import tqdm
import numpy as np

In [25]:
# set running device to cpu

device = "cpu"

In [26]:
# see openai-clip available pre-train model

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [27]:
# load Vit-B/32 model

model, preprocess = clip.load("ViT-B/32", device=device)

In [28]:
# load dataset

datasetFolder_path = os.getcwd() + '/selected_images/'

In [29]:
# create list of all filename in dataset folder

imageFilename_list = os.listdir(datasetFolder_path)

In [30]:
# encode dataset
datasetEmbeddings_list = []

for imageFilename in tqdm(imageFilename_list):
    with torch.no_grad():
        image = preprocess(Image.open(datasetFolder_path + imageFilename)).unsqueeze(0).to(device)
        datasetEmbeddings_list.append(model.encode_image(image))


  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:03<00:00,  5.38it/s]


In [ ]:
import gradio as gr

def find_similar_image(imageFile_path):
    print(f"get image path {imageFile_path}")

    similarityScore_list = []
    test_image = preprocess(Image.open(imageFile_path)).unsqueeze(0).to(device)

    with torch.no_grad():
        test_embeddings = model.encode_image(test_image)

    for index, dataset_embeddings in enumerate(datasetEmbeddings_list):
        score = torch.nn.functional.cosine_similarity(test_embeddings, dataset_embeddings)
        # print(f"similarity to {imageFilename_list[index]} : {score.item()}")
        similarityScore_list.append(score.item())

    # find max similarity
    max_score = np.max(similarityScore_list)
    # print(f"max similarity {np.max(similarityScore_list)}")

    max_index = np.where(similarityScore_list == max_score)[0][0]
    # print(f"max index {max_index}")
    recommend_image = datasetFolder_path + imageFilename_list[max_index]

    return recommend_image

In [ ]:
demo = gr.Interface(
    fn=find_similar_image,
    inputs=gr.Image(type="filepath"),
    outputs= gr.Image(type="filepath"),
)

In [ ]:
demo.launch()